In [ ]:
!pip install lxml
!pip install requests
!pip install beautifulsoup4
import pandas as pd
from bs4 import BeautifulSoup
import csv
import requests
import json
import re
import os

In [ ]:
def get_blog_url(soup):
  div_=soup.find_all('div', attrs={'class': 'FL PR20'})
  url_list=[]
  for title in div_:
    href =title.find('a')['href']
    url_list.append("https://www.moneycontrol.com/"+href)
  return url_list

In [ ]:
def get_blog_content(url):
  request= requests.get(url)
  soup =BeautifulSoup(request.text, 'lxml')
  all_scripts= soup.find_all('script', attrs= {'type':'application/ld+json'})
  raw_article_str=all_scripts[2].get_text().replace('\r\n','')
  parts= re.split(r"""("[^"]*"|'[^']*')""", raw_article_str)
  parts[::2] = map(lambda s: "".join(s.split()), parts[::2])
  article_str="".join(parts)
  article_str= article_str[1:]
  article_str= article_str[:-1]
  article_dict= json.loads(article_str)
  all_tags=soup.find_all('div', attrs= {'class':'tags_first_line'})
  lst_all_tags=[]
  for i in all_tags:
    lst_all_tags.append(i.get_text())
  tags= lst_all_tags[0].replace('TAGS:','')
  tags=tags.replace('\n','')
  tags=tags.split('#')
  tags= tags[1:]
  tags=','.join([str(elem).strip() for elem in tags])
  article_dict['tags']=tags
  return article_dict

In [ ]:
url="https://www.moneycontrol.com/news/telecom/telecom-sector-in-2017-a-year-marked-by-bitter-competition-and-consolidation-2469671.html"
vl=get_blog_content(url)
print(vl)

FeatureNotFound: Couldn't find a tree builder with the features you requested: htal.parser. Do you need to install a parser library?

In [ ]:
def get_page_no(url, sc_id, page_no, next, year):
  request= requests.get(url)
  soup =BeautifulSoup(request.text, 'lxml')
  all_page_no =soup.find_all('div', attrs={'class': 'pages MR10 MT15'})
  page_list =[i.text for i in all_page_no[0].find_all('a')]
  if any(map(str.isdigit, page_list[-1])):

    return int(page_list[-1]), next

  else:
    next= next + 1
    page_no =int(page_list[-2])
    url="https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id="+sc_id+"&scat=&pageno="+str(page_no)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
    return get_page_no(url, sc_id, page_no, next, year)

In [ ]:
def save_company_data(url_="https://www.moneycontrol.com/stocks/company_info/stock_news.php?", sc_id=[], page_no=1, next=0, years=[]):
  for company in sc_id:
    df=pd.DataFrame(columns=['company', 'datePublished', 'author', 'headline', 'description', 'articleBody', 'tags', 'url'])
    for year in years:
      print('year:', year)
      print('page_no:',page_no)
      print('next:', next)
      url=url_ + "sc id="+company+"&scat=&pageno="+str(page_no)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
      print('url:', url)

      max_page_no, max_next= get_page_no(url, company, page_no, next, year)
      max_next= max_next + 1

      for i in range(max_next):
        for j in range((i*10)+1, (i*10)+11):
          if j <= max_page_no:
            url_list=[]
            url=url_ + "sc_id="+company+"&scat=&pageno="+str(j)+"&next="+str(i)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
            request= requests.get(url)
            soup= BeautifulSoup(request.text, 'lxml')
            url_list= get_blog_url(soup)

            for url in url_list:
              try:
                article_dict= get_blog_content(url)
                print(company)
                print(article_dict['datePublished'])
                print(article_dict['author'])
                print(article_dict['headline'])
                print(article_dict['description'])
                print(article_dict['articleBody'])
                print(article_dict['tags'])
                print(article_dict['url'])
                print('------------------------------------------')

                article_lst =[[company,
                              article_dict['datePublished'],
                              article_dict['author'],
                              article_dict['headline'],
                              article_dict['description'],
                              article_dict['articleBody'],
                              article_dict['tags'], url]]


                df = pd.concat([df, pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline', 'description', 'articleBody', 'tags', 'url'])], ignore_index=True)



              except:

                   article_lst =[[company, 'error', 'error', 'error', 'error', 'error', 'error', url]]

                   df = pd.concat([df, pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline', 'description', 'articleBody', 'tags', 'url'])], ignore_index=True)

                   continue
          else:
                 break
  df.to_csv('/content/drive/MyDrive/ALLM_Project/'+company+'.csv')

In [ ]:
print(save_company_data(sc_id=["RI"], page_no=1, next=0, years=[2017]))

year: 2017
page_no: 1
next: 0
url: https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc id=RI&scat=&pageno=1&next=0&durationType=Y&Year=2017&duration=1&news_type=
RI
2017-12-29T16:56:24+05:30
{'@type': 'Person', 'url': 'https://www.moneycontrol.com/author/nachiket-deuskar-7511/', 'name': 'Nachiket Deuskar'}
Telecom sector in 2017: A year marked by bitter competition and consolidation
In what could be considered a landmark year for the telecom sector in India, 2017 saw telcos consolidate like never before. By the end of the year, there were only three major players left in the market. 
Moneycontrol NewsOn Tuesday, Anil Ambani-led Reliance Communications said that the telecom sector has become a money guzzler where only those with deep pockets can survive.Even the &amp;quot;mighty&amp;quot; house of Tatas had to &amp;quot;gift away&amp;quot; their telecom business (to Airtel), Ambani asserted, sounding bitter about the regulatory framework. He said the long time taken to cl

In [ ]:
df= df.append(pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline',
                                                                 'description', 'articleBody', 'tags', 'url']), ignore_index= True)
df=df.append(pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline',
                   'description', 'articleBody', 'tags', 'url']), ignore_index= True)

In [ ]:
print(save_company_data(sc_id=["RI"], page_no=1, next=0, years=[2017]))

year: 2017
page_no: 1
next: 0
url: https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc id=RI&scat=&pageno=1&next=0&durationType=Y&Year=2017&duration=1&news_type=
None


In [ ]:
sc_id="RI"
page_no=1
next=0
year=2017
url="https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id="+sc_id+"&scat=&pageno="+str(page_no)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
value4=get_page_no(url, sc_id, page_no, next, year)

In [ ]:
print(value4)

(21, 2, 'https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=RI&scat=&pageno=20&next=2&durationType=Y&Year=2017&duration=1&news_type=', 'RI', 2017)


In [ ]:
!pip install fpdf

import csv
from fpdf import FPDF

def csv_to_row_pdf(csv_file, pdf_file):
  """
  Converts a CSV file with columns into rows and saves it as a PDF document.

  Args:
      csv_file (str): Path to the CSV file.
      pdf_file (str): Path to the output PDF file.
  """
  # Open the CSV file
  with open(csv_file, 'r') as csvfile:
    reader = csv.reader(csvfile)

    # Create a new PDF document
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=10)
    cell_width = 100
    # Loop through each row in the CSV file
    for row in reader:
      # Write each element in the row as a separate line in the PDF
      for cell in row:
        pdf.cell(w=cell_width, h=5, txt=cell, ln=1)  # Add cell content and move to next line

    # Save the PDF document
    pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI.csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/data_rows.pdf'
csv_to_row_pdf(csv_file, pdf_file)


In [35]:
import csv
import os
from fpdf import FPDF

def csv_to_row_pdf(csv_file, pdf_file):
  """
  Converts a CSV file with rows into separate rows (including nested elements) and saves it as a PDF document, skipping the header row.

  Args:
      csv_file (str): Path to the CSV file.
      pdf_file (str): Path to the output PDF file.
  """
  # Create temporary CSV without header (optional)
  with open(csv_file, 'r') as infile, open('temp_data.csv', 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    next(reader)
    for row in reader:
      writer.writerow(row[1:])

  # Read data from temporary CSV (modify path if needed)
  with open('temp_data.csv', 'r',encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)  # Adjust if using nested elements

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=10)
    cell_width = 0

    for row in reader:

      # ... rest of the code to iterate through rows and create PDF content
      for cell in row:
        pdf.cell(w=cell_width, h=5, txt=cell, ln=1)  # Add cell content and move to next line

    # Save the PDF document
    pdf.output(pdf_file)
  # Delete temporary CSV (optional)

  os.remove('temp_data.csv')

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI_main.csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/a.pdf'
csv_to_row_pdf(csv_file, pdf_file)


<ipython-input-35-18627a85fdd8>:27: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=10)
<ipython-input-35-18627a85fdd8>:34: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(w=cell_width, h=5, txt=cell, ln=1)  # Add cell content and move to next line
<ipython-input-35-18627a85fdd8>:34: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(w=cell_width, h=5, txt=cell, ln=1)  # Add cell content and move to next line


In [36]:
import pandas as pd
from fpdf import FPDF

def csv_to_row_pdf_pandas(csv_file, pdf_file):
  """
  Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document, skipping the header row and index.

  Args:
      csv_file (str): Path to the CSV file.
      pdf_file (str): Path to the output PDF file.
  """

  # Read CSV data into a pandas DataFrame
  df = pd.read_csv(csv_file)

  # Skip header row (if present) and index column
  df = df.iloc[:, 1:]  # Select rows from index 1 (skipping header) and column index 1 (skipping index)

  # Create FPDF object and set formatting
  pdf = FPDF()
  pdf.add_page()
  pdf.set_font("Arial", size=10)
  cell_width = 0

  # Iterate through rows (converted to Series) and write to PDF
  for row in df.itertuples(index=False):
    # Iterate through key-value pairs (column names and values)
    for col, value in zip(df.columns, row):
        # Skip 'url' column (if needed)
        pdf.cell(w=cell_width, h=5, txt=f"{col}: {value}", ln=1)

  # Save the PDF document
  pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI_main.csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/a.pdf'
csv_to_row_pdf_pandas(csv_file, pdf_file)


<ipython-input-36-ee810a8ddf89>:22: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=10)
<ipython-input-36-ee810a8ddf89>:30: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(w=cell_width, h=5, txt=f"{col}: {value}", ln=1)
<ipython-input-36-ee810a8ddf89>:30: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(w=cell_width, h=5, txt=f"{col}: {value}", ln=1)


FPDFUnicodeEncodingException: Character "’" at index 33 in text is outside the range of characters supported by the font used: "helvetica". Please consider using a Unicode font.

In [31]:
import pandas as pd
from fpdf import FPDF

def csv_to_row_pdf_pandas(csv_file, pdf_file):
  """
  Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document, skipping the header row and index.

  Args:
      csv_file (str): Path to the CSV file.
      pdf_file (str): Path to the output PDF file.
  """

  # Read CSV data into a pandas DataFrame
  df = pd.read_csv(csv_file)

  # Skip header row (if present) and keep the first row of data
  df = df.iloc[:, 1:]  # Select all rows and skip only the index column

  # Create FPDF object and set formatting
  pdf = FPDF()
  pdf.add_page()
  pdf.set_font("Arial", size=10)
  cell_width = 1000

  # Iterate through rows (converted to Series) and write to PDF
  for row in df.itertuples(index=False):
    # Convert row to string to handle nested elements
    row_str = str(row)
    # Replace unwanted characters from the string representation of the row
    row_str = row_str.replace("(", "").replace(")", "").replace("'", "")
    # Split the row string by commas to get individual cell values
    cells = row_str.split(", ")
    for cell in cells:
        # Write each cell to the PDF, adjusting cell width as needed
        pdf.cell(w=cell_width, h=5, txt=cell, ln=1)

  # Save the PDF document
  pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI (4).csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/data4.pdf'
csv_to_row_pdf_pandas(csv_file, pdf_file)


<ipython-input-31-f6696c831302>:22: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=10)
<ipython-input-31-f6696c831302>:35: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(w=cell_width, h=5, txt=cell, ln=1)
<ipython-input-31-f6696c831302>:35: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(w=cell_width, h=5, txt=cell, ln=1)


FPDFUnicodeEncodingException: Character "’" at index 32 in text is outside the range of characters supported by the font used: "helvetica". Please consider using a Unicode font.

In [37]:
import pandas as pd
from fpdf import FPDF

def csv_to_row_pdf_pandas(csv_file, pdf_file):
    """
    Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document, skipping the header row and index.

    Args:
        csv_file (str): Path to the CSV file.
        pdf_file (str): Path to the output PDF file.
    """

    # Read CSV data into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Create FPDF object and set formatting
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=10)
    cell_width = 0  # Auto width

    # Iterate through rows and write to PDF
    for index, row in df.iterrows():
        for col, value in row.items():
            # Check if value is a nested element (e.g., list or dict)
            if isinstance(value, (list, dict)):
                # Convert nested elements to string
                value = ', '.join([str(elem) for elem in value]) if isinstance(value, list) else str(value)
            # Write to PDF, adjust cell height as needed
            pdf.multi_cell(w=cell_width, h=10, txt=f"{col}: {value}")
        pdf.ln(10)  # Add space between rows

    # Save the PDF document
    pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI_main.csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/a.pdf'
csv_to_row_pdf_pandas(csv_file, pdf_file)



<ipython-input-37-0714a47b8740>:19: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=10)
<ipython-input-37-0714a47b8740>:30: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.multi_cell(w=cell_width, h=10, txt=f"{col}: {value}")


FPDFException: Not enough horizontal space to render a single character

In [43]:
import pandas as pd
!pip install reportlab

from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors

def csv_to_row_pdf_pandas(csv_file, pdf_file):
    """
    Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document,
    skipping the header row and index.

    Args:
        csv_file (str): Path to the CSV file.
        pdf_file (str): Path to the output PDF file.
    """

    # Read CSV data into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Create a list to hold table data
    table_data = []

    # Iterate through rows and add data to the table
    for index, row in df.iterrows():
        if index == 0:
            continue  # Skip header row
        table_data.append([f"{col}: {value}" for col, value in row.items()])

    # Create a PDF document
    doc = SimpleDocTemplate(pdf_file, pagesize=letter)
    table = Table(table_data, colWidths=[cell_width] * len(df.columns))
    table.setStyle(TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                               ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                               ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                               ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                               ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                               ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                               ('GRID', (0, 0), (-1, -1), 1, colors.black)]))

    # Build the PDF
    doc.build([table])

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI_main.csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/a.pdf'
cell_width = 100  # Adjust cell width as needed
csv_to_row_pdf_pandas(csv_file, pdf_file)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.1 MB/s eta 0:00:00


In [29]:
import pandas as pd
!pip install fpdf
from fpdf import FPDF

def csv_to_row_pdf_pandas(csv_file, pdf_file):
    """
    Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document.

    Args:
        csv_file (str): Path to the CSV file.
        pdf_file (str): Path to the output PDF file.
    """

    # Read CSV data into a pandas DataFrame, using the first column as the index
    df = pd.read_csv(csv_file, index_col=0)

    # Create FPDF object and set formatting
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=10)
    cell_width = 190  # Auto width

    # Iterate through rows and write to PDF
    for index, row in df.iterrows():
        for col, value in row.items():
            # Check if value is a nested element (e.g., list or dict)
            if isinstance(value, (list, dict)):
                # Convert nested elements to string
                value = ', '.join([str(elem) for elem in value]) if isinstance(value, list) else str(value)
            # Write to PDF, adjust cell height as needed
            pdf.multi_cell(w=cell_width, h=10, txt=f"{col}: {value}")
        pdf.ln(10)  # Add space between rows

    # Save the PDF document
    pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI (4).csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/data.pdf'
csv_to_row_pdf_pandas(csv_file, pdf_file)


<ipython-input-29-1fa4b09a4bc5>:20: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=10)
<ipython-input-29-1fa4b09a4bc5>:31: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.multi_cell(w=cell_width, h=10, txt=f"{col}: {value}")


FPDFUnicodeEncodingException: Character "’" at index 33 in text is outside the range of characters supported by the font used: "helvetica". Please consider using a Unicode font.

In [24]:
import pandas as pd
from fpdf import FPDF

def csv_to_row_pdf_pandas(csv_file, pdf_file):
    """
    Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document.

    Args:
        csv_file (str): Path to the CSV file.
        pdf_file (str): Path to the output PDF file.
    """

    # Read CSV data into a pandas DataFrame, using the first column as the index
    df = pd.read_csv(csv_file, index_col=0)

    # Create FPDF object and set formatting
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=10)
    cell_width = 190  # Set a specific width for the cell

    # Iterate through rows and write to PDF
    for index, row in df.iterrows():
        for col, value in row.items():
            # Check if value is a nested element (e.g., list or dict)
            if isinstance(value, (list, dict)):
                # Convert nested elements to string
                value = ', '.join([str(elem) for elem in value]) if isinstance(value, list) else str(value)
            # Write to PDF, adjust cell height as needed
            pdf.multi_cell(w=cell_width, h=10, txt=f"{col}: {value}")
        pdf.ln(10)  # Add space between rows

    # Save the PDF document
    pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI_M14.csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/dataM.pdf'
csv_to_row_pdf_pandas(csv_file, pdf_file)


<ipython-input-24-c1d580af4b5a>:19: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=10)
<ipython-input-24-c1d580af4b5a>:30: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.multi_cell(w=cell_width, h=10, txt=f"{col}: {value}")


FPDFUnicodeEncodingException: Character "’" at index 33 in text is outside the range of characters supported by the font used: "helvetica". Please consider using a Unicode font.

In [10]:
import pandas as pd
from fpdf import FPDF

def csv_to_row_pdf_pandas(csv_file, pdf_file):
    """
    Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document.

    Args:
        csv_file (str): Path to the CSV file.
        pdf_file (str): Path to the output PDF file.
    """

    # Read CSV data into a pandas DataFrame, using the first column as the index
    df = pd.read_csv(csv_file, index_col=0)

    # Create FPDF object and set formatting
    pdf = FPDF()
    pdf.add_page()

    # Add a Unicode font (e.g., DejaVu)
    pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
    pdf.set_font('DejaVu', '', 12)

    # Set a specific width for the cell if needed
    cell_width = 190

    # Iterate through rows and write to PDF
    for index, row in df.iterrows():
        for col, value in row.items():
            # Check if value is a nested element (e.g., list or dict)
            if isinstance(value, (list, dict)):
                # Convert nested elements to string
                value = ', '.join([str(elem) for elem in value]) if isinstance(value, list) else str(value)
            # Write to PDF, adjust cell height as needed
            pdf.multi_cell(w=cell_width, h=10, txt=f"{col}: {value}")
        pdf.ln(10)  # Add space between rows

    # Save the PDF document
    pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI_M1.csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/data.pdf'
csv_to_row_pdf_pandas(csv_file, pdf_file)


<ipython-input-10-f99f77cb0593>:21: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
<ipython-input-10-f99f77cb0593>:35: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.multi_cell(w=cell_width, h=10, txt=f"{col}: {value}")


In [15]:
import pandas as pd
from fpdf import FPDF

# Function to convert CSV to PDF
def csv_to_row_pdf_pandas(csv_file, pdf_file):
    """
    Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document.

    Args:
        csv_file (str): Path to the CSV file.
        pdf_file (str): Path to the output PDF file.
    """

    # Read CSV data into a pandas DataFrame, using the first column as the index
    df = pd.read_csv(csv_file, index_col=0)

    # Create FPDF object and set formatting
    pdf = FPDF()
    pdf.add_page()

    # Add a Unicode font (e.g., DejaVu)
    pdf.add_font('ArialUnicode', '', '/content/drive/MyDrive/ALLM_Project/arial-unicode-ms.ttf', uni=True)
    pdf.set_font('ArialUnicode', '', 12)

    # Set a specific width for the cell if needed
    cell_width = 190

    # Iterate through rows and write to PDF
    for index, row in df.iterrows():
        for col, value in row.items():
            # Check if value is a nested element (e.g., list or dict)
            if isinstance(value, (list, dict)):
                # Convert nested elements to string
                value = ', '.join([str(elem) for elem in value]) if isinstance(value, list) else str(value)
            # Write to PDF, adjust cell height as needed
            pdf.multi_cell(w=cell_width, h=10, text=f"{col}: {value}")
        pdf.ln(10)  # Add space between rows

    # Save the PDF document
    pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI_M1.csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/data.pdf'
csv_to_row_pdf_pandas(csv_file, pdf_file)


<ipython-input-15-40bafbd8e575>:22: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('ArialUnicode', '', '/content/drive/MyDrive/ALLM_Project/arial-unicode-ms.ttf', uni=True)


In [18]:
!pip install tfpdf

from tpdf import TPDF
# Function to convert CSV to PDF
def csv_to_row_pdf_pandas(csv_file, pdf_file):
    """
    Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document.

    Args:
        csv_file (str): Path to the CSV file.
        pdf_file (str): Path to the output PDF file.
    """

    # Read CSV data into a pandas DataFrame, using the first column as the index
    df = pd.read_csv(csv_file, index_col=0)

    # Create FPDF object and set formatting
    pdf = TPDF()
    pdf.add_page()

    # Add a Unicode font (e.g., DejaVu)
    pdf.add_font('ArialUnicode', '', '/content/drive/MyDrive/ALLM_Project/arial-unicode-ms.ttf', uni=True)
    pdf.set_font('ArialUnicode', '', 12)

    # Set a specific width for the cell if needed
    cell_width = 190

    # Iterate through rows and write to PDF
    for index, row in df.iterrows():
        for col, value in row.items():
            # Check if value is a nested element (e.g., list or dict)
            if isinstance(value, (list, dict)):
                # Convert nested elements to string
                value = ', '.join([str(elem) for elem in value]) if isinstance(value, list) else str(value)
            # Write to PDF, adjust cell height as needed
            pdf.multi_cell(w=cell_width, h=10, text=f"{col}: {value}")
        pdf.ln(10)  # Add space between rows

    # Save the PDF document
    pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI_M1.csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/dataa.pdf'
csv_to_row_pdf_pandas(csv_file, pdf_file)

ERROR: Could not find a version that satisfies the requirement tfpdf (from versions: none)
ERROR: No matching distribution found for tfpdf


ModuleNotFoundError: No module named 'tpdf'

In [22]:
!pip install fpdf2

import pandas as pd
from fpdf import FPDF  # Import FPDF from fpdf2

# Function to convert CSV to PDF
def csv_to_row_pdf_pandas(csv_file, pdf_file):
    """
    Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document.

    Args:
        csv_file (str): Path to the CSV file.
        pdf_file (str): Path to the output PDF file.
    """

    # Read CSV data into a pandas DataFrame, using the first column as the index
    df = pd.read_csv(csv_file, index_col=0)

    # Create FPDF object and set formatting
    pdf = FPDF()
    pdf.add_page()

    # Add a Unicode font (e.g., ArialUnicode)
    pdf.add_font('ArialUnicode', '', '/content/drive/MyDrive/ALLM_Project/arial-unicode-ms.ttf', uni=True)
    pdf.set_font('ArialUnicode', '', 12)

    # Set a specific width for the cell if needed
    cell_width = 190

    # Iterate through rows and write to PDF
    for index, row in df.iterrows():
        for col, value in row.items():
            # Check if value is a nested element (e.g., list or dict)
            if isinstance(value, (list, dict)):
                # Convert nested elements to string
                value = ', '.join([str(elem) for elem in value]) if isinstance(value, list) else str(value)
            # Write to PDF, adjust cell height as needed
            pdf.multi_cell(w=cell_width, h=10, text=f"{col}: {value}")
        pdf.ln(10)  # Add space between rows

    # Save the PDF document
    pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI_M1.csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/dataa.pdf'
csv_to_row_pdf_pandas(csv_file, pdf_file)


<ipython-input-22-e30a2d06620f>:24: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('ArialUnicode', '', '/content/drive/MyDrive/ALLM_Project/arial-unicode-ms.ttf', uni=True)


In [26]:
!pip install fpdf2

import pandas as pd
from fpdf import FPDF  # Import FPDF from fpdf2

# Function to convert CSV to PDF
def csv_to_row_pdf_pandas(csv_file, pdf_file, fallback_fonts):
    """
    Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document.
    Implements a font fallback mechanism for missing glyphs.

    Args:
        csv_file (str): Path to the CSV file.
        pdf_file (str): Path to the output PDF file.
        fallback_fonts (list): List of tuples with fallback font details.
    """

    # Read CSV data into a pandas DataFrame, using the first column as the index
    df = pd.read_csv(csv_file, index_col=0)

    # Create FPDF object and set formatting
    pdf = FPDF()
    pdf.add_page()

    # Add primary Unicode font (e.g., ArialUnicode)
    pdf.add_font('ArialUnicode', '', '/content/drive/MyDrive/ALLM_Project/arial-unicode-ms.ttf', uni=True)

    # Add fallback fonts
    for font_name, font_path in fallback_fonts:
        pdf.add_font(font_name, '', font_path, uni=True)

    # Function to set font with fallback
    def set_font_with_fallback(pdf, text):
        for font_name, _ in fallback_fonts:
            pdf.set_font(font_name, '', 12)
            if pdf.get_string_width(text) != 0:  # If the string width is not 0, the font can display the text
                return
        pdf.set_font('ArialUnicode', '', 12)  # Default to primary font if no fallback font works

    # Set a specific width for the cell if needed
    cell_width = 190

    # Iterate through rows and write to PDF
    for index, row in df.iterrows():
        for col, value in row.items():
            # Check if value is a nested element (e.g., list or dict)
            if isinstance(value, (list, dict)):
                # Convert nested elements to string
                value = ', '.join([str(elem) for elem in value]) if isinstance(value, list) else str(value)
            # Set font with fallback before writing
            set_font_with_fallback(pdf, f"{col}: {value}")
            # Write to PDF, adjust cell height as needed
            pdf.multi_cell(w=cell_width, h=10, text=f"{col}: {value}")
        pdf.ln(10)  # Add space between rows

    # Save the PDF document
    pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI_M1.csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/dataa.pdf'
fallback_fonts = [('DejaVuSans', '/content/drive/MyDrive/ALLM_Project/DejaVuSans.ttf')]  # Add paths to your fallback fonts here
csv_to_row_pdf_pandas(csv_file, pdf_file, fallback_fonts)


<ipython-input-26-5a1b5e55e29e>:26: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('ArialUnicode', '', '/content/drive/MyDrive/ALLM_Project/arial-unicode-ms.ttf', uni=True)
<ipython-input-26-5a1b5e55e29e>:30: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font(font_name, '', font_path, uni=True)


In [27]:
!pip install fpdf2

import pandas as pd
from fpdf import FPDF

# Custom FPDF class to ignore missing glyphs warning
class MyFPDF(FPDF):
    def error(self, msg):
        if 'missing the following glyphs' in msg:
            pass  # Ignore missing glyphs warning
        else:
            super().error(msg)

# Function to convert CSV to PDF
def csv_to_row_pdf_pandas(csv_file, pdf_file, font_path):
    """
    Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document.
    Ignores missing glyphs warning.

    Args:
        csv_file (str): Path to the CSV file.
        pdf_file (str): Path to the output PDF file.
        font_path (str): Path to the font file.
    """

    # Read CSV data into a pandas DataFrame, using the first column as the index
    df = pd.read_csv(csv_file, index_col=0)

    # Create MyFPDF object and set formatting
    pdf = MyFPDF()
    pdf.add_page()

    # Add a Unicode font (e.g., ArialUnicode)
    pdf.add_font('ArialUnicode', '', font_path, uni=True)
    pdf.set_font('ArialUnicode', '', 12)

    # Set a specific width for the cell if needed
    cell_width = 190

    # Iterate through rows and write to PDF
    for index, row in df.iterrows():
        for col, value in row.items():
            # Check if value is a nested element (e.g., list or dict)
            if isinstance(value, (list, dict)):
                # Convert nested elements to string
                value = ', '.join([str(elem) for elem in value]) if isinstance(value, list) else str(value)
            # Write to PDF, adjust cell height as needed
            pdf.multi_cell(w=cell_width, h=10, text=f"{col}: {value}")
        pdf.ln(10)  # Add space between rows

    # Save the PDF document
    pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI_M1.csv'  # Replace with your CSV file path
pdf_file = '/content/drive/MyDrive/ALLM_Project/dataa4.pdf'  # Replace with your output PDF file path
font_path = '/content/drive/MyDrive/ALLM_Project/arial-unicode-ms.ttf'  # Replace with your font file path
csv_to_row_pdf_pandas(csv_file, pdf_file, font_path)


<ipython-input-27-544b3a7a74ce>:34: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('ArialUnicode', '', font_path, uni=True)


In [28]:
!pip install fpdf2

import pandas as pd
from fpdf import FPDF

# Custom FPDF class to ignore missing glyphs warning
class MyFPDF(FPDF):
    def error(self, msg):
        if 'missing the following glyphs' in msg:
            pass  # Ignore missing glyphs warning
        else:
            super().error(msg)

# Function to sanitize data by replacing or removing problematic characters
def sanitize_data(value):
    # Define a mapping of problematic characters to their replacements
    replacements = {
        '\u0080': '',  # Replace with an empty string or a suitable placeholder
        '\u0093': '',
        '\u0099': '',
        '\uFEFF': '',
        '\u009D': ''
    }
    for original, replacement in replacements.items():
        value = value.replace(original, replacement)
    return value

# Function to convert CSV to PDF
def csv_to_row_pdf_pandas(csv_file, pdf_file, font_path):
    """
    Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document.
    Ignores missing glyphs warning and sanitizes data.

    Args:
        csv_file (str): Path to the CSV file.
        pdf_file (str): Path to the output PDF file.
        font_path (str): Path to the font file.
    """

    # Read CSV data into a pandas DataFrame, using the first column as the index
    df = pd.read_csv(csv_file, index_col=0)

    # Create MyFPDF object and set formatting
    pdf = MyFPDF()
    pdf.add_page()

    # Add a Unicode font (e.g., ArialUnicode)
    pdf.add_font('ArialUnicode', '', font_path, uni=True)
    pdf.set_font('ArialUnicode', '', 12)

    # Set a specific width for the cell if needed
    cell_width = 190

    # Iterate through rows and write to PDF
    for index, row in df.iterrows():
        for col, value in row.items():
            # Sanitize data to remove or replace problematic characters
            value = sanitize_data(str(value))
            # Write to PDF, adjust cell height as needed
            pdf.multi_cell(w=cell_width, h=10, text=f"{col}: {value}")
        pdf.ln(10)  # Add space between rows

    # Save the PDF document
    pdf.output(pdf_file)

# Example usage
csv_file = '/content/drive/MyDrive/ALLM_Project/RI_M1.csv'  # Replace with your CSV file path
pdf_file = '/content/drive/MyDrive/ALLM_Project/dataa4.pdf'  # Replace with your output PDF file path
font_path = '/content/drive/MyDrive/ALLM_Project/arial-unicode-ms.ttf'  # Replace with your font file path
csv_to_row_pdf_pandas(csv_file, pdf_file, font_path)


<ipython-input-28-2b3825d9ed09>:48: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('ArialUnicode', '', font_path, uni=True)


In [1]:
!pip install fpdf2
import pandas as pd
from fpdf import FPDF

# If you haven't already, you may need to install fpdf2:
# pip install fpdf2

from fpdf import FPDF, UTF8

def csv_to_row_pdf_pandas(csv_file, pdf_file):
    """
    Converts a CSV file with columns into separate rows (including nested elements) and saves it as a PDF document.

    Args:
        csv_file (str): Path to the CSV file.
        pdf_file (str): Path to the output PDF file.
    """

    # Read CSV data into a pandas DataFrame, using the first column as the index
    df = pd.read_csv(csv_file, index_col=0)

    # Create FPDF object with UTF-8 encoding
    pdf = FPDF()
    pdf.set_doc_option('core_fonts_encoding', 'utf-8')
    pdf.add_page()
    pdf.set_font("Arial", size=10)
    cell_width = 0  # Auto width

    # Iterate through rows and write to PDF
    for index, row in df.iterrows():
        for col, value in row.items():
            # Check if value is a nested element (e.g., list or dict)
            if isinstance(value, (list, dict)):
                # Convert nested elements to string
                value = ', '.join([str(elem) for elem in value]) if isinstance(value, list) else str(value)
            # Write to PDF, adjust cell height as needed
            pdf.multi_cell(w=cell_width, h=10, txt=f"{col}: {value}", align='L')
        pdf.ln(10)  # Add space between rows

    # Save the PDF document
    pdf.output(pdf_file)

# Example usage
csv_file = '/content/RI_M.csv'
pdf_file = '/content/drive/MyDrive/ALLM_Project/data.pdf'
csv_to_row_pdf_pandas(csv_file, pdf_file)


/usr/local/lib/python3.10/dist-packages/fpdf/__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


ImportError: cannot import name 'UTF8' from 'fpdf' (/usr/local/lib/python3.10/dist-packages/fpdf/__init__.py)

In [ ]:
# Install pdfkit
!pip install pdfkit

# Install wkhtmltopdf
!apt-get install wkhtmltopdf

# Import necessary libraries
import pandas as pd
import pdfkit

# Load your CSV file
df = pd.read_csv('/content/drive/MyDrive/ALLM_Project/RI_main.csv')

# Convert the DataFrame to HTML format
html_string = df.to_html(index=False)

# Convert the HTML string to a PDF file
pdfkit.from_string(html_string, 'output_file.pdf')

# If you want to download the file to your local system
from google.colab import files
files.download('output_file.pdf')


In [66]:
# Install the FPDF library
!pip install fpdf

# Import the library
from fpdf import FPDF
import pandas as pd

# Define a class to create the PDF
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'CSV Data', 0, 1, 'C')

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

# Load your CSV file
df = pd.read_csv('/content/drive/MyDrive/ALLM_Project/RI_M2.csv')

# Create instance of FPDF class
pdf = PDF()

# Add a page
pdf.add_page()

pdf.set_font("Arial", size=12)

# Add a cell
for index, row in df.iterrows():
    pdf.cell(200, 10, txt=f"Company: {row['company']}", ln=True)
    pdf.cell(200, 10, txt=f"Date Published: {row['datePublished']}", ln=True)
    pdf.cell(200, 10, txt=f"Author: {row['author']}", ln=True)
    pdf.cell(200, 10, txt=f"Headline: {row['headline']}", ln=True)
    pdf.cell(200, 10, txt=f"Description: {row['description']}", ln=True)
    pdf.multi_cell(200, 10, txt=f"Article Body: {row['articleBody']}", ln=True)
    #pdf.cell(200, 10, txt=f"Tags: {row['tags']}", ln=True)
    pdf.cell(200, 10, txt=f"URL: {row['url']}", ln=True)
    pdf.cell(200, 10, txt="", ln=True)  # Add an empty line for spacing

# Save the pdf with name .pdf
pdf.output("output_file.pdf")

# If you want to download the file to your local system
from google.colab import files
files.download('output_file.pdf')


<ipython-input-66-ee886cdefd92>:11: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
<ipython-input-66-ee886cdefd92>:12: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'CSV Data', 0, 1, 'C')
<ipython-input-66-ee886cdefd92>:28: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=12)
<ipython-input-66-ee886cdefd92>:32: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt=f"Company: {row['company']}", ln=True)
<ipython-input-66-ee886cdefd92>:32: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt=f"Company: {row['company']}", ln=True)
<ipy

FPDFUnicodeEncodingException: Character "‐" at index 835 in text is outside the range of characters supported by the font used: "helvetica". Please consider using a Unicode font.

In [53]:
# Install the FPDF library
!pip install fpdf

# Import the library
from fpdf import FPDF
import pandas as pd

# Define a class to create the PDF
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, '', 0, 1, 'C')

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

# Load your CSV file
df = pd.read_csv('/content/drive/MyDrive/ALLM_Project/RI_M1.csv')

# Create instance of FPDF class
pdf = PDF()

# Add a page
pdf.add_page()

# Set font to Arial, 'B'old, 12 pts
pdf.set_font('Arial', '', 12)

# Add Unicode font support
pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
pdf.set_font('DejaVu', '', 14)

# Add a cell
for index, row in df.iterrows():
    pdf.cell(200, 10, txt=f"Company: {row['company']}", ln=True)
    pdf.cell(200, 10, txt=f"Date Published: {row['datePublished']}", ln=True)
    pdf.cell(200, 10, txt=f"Author: {row['author']}", ln=True)
    pdf.cell(200, 10, txt=f"Headline: {row['headline']}", ln=True)
    pdf.cell(200, 10, txt=f"Description: {row['description']}", ln=True)
    pdf.cell(200, 10, txt=f"Article Body: {row['articleBody']}", ln=True)
    pdf.cell(200, 10, txt=f"Tags: {row['tags']}", ln=True)
    pdf.cell(200, 10, txt=f"URL: {row['url']}", ln=True)
    pdf.cell(200, 10, txt="", ln=True)  # Add an empty line for spacing

# Save the pdf with name .pdf
pdf.output("output_file.pdf")

# If you want to download the file to your local system
from google.colab import files
files.download('output_file.pdf')


<ipython-input-53-e3e3164ea7ac>:11: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
<ipython-input-53-e3e3164ea7ac>:12: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, '', 0, 1, 'C')
<ipython-input-53-e3e3164ea7ac>:29: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 12)
<ipython-input-53-e3e3164ea7ac>:32: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
<ipython-input-53-e3e3164ea7ac>:37: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(600, 10, txt=f"Company: {row['company']}", ln=True)
<ipyt

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [60]:
!pip install fpdf

from fpdf import FPDF
import pandas as pd

# Define a class to create the PDF
class PDF(FPDF):
  def header(self):
    self.set_font('Arial', 'B', 12)
    self.cell(0, 10, '', 0, 1, 'C')

  def footer(self):
    self.set_y(-15)
    self.set_font('Arial', 'I', 8)
    self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

# Load your CSV file
df = pd.read_csv('/content/drive/MyDrive/ALLM_Project/RI_M1.csv')

# Create instance of FPDF class
pdf = PDF()

# Add a page
pdf.add_page()

# Set font to Arial, '', 12 pts
pdf.set_font('Arial', '', 8)

# Add Unicode font support for special characters
pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
pdf.set_font('DejaVu', '', 8)

# Increase cell width to fit more text
cell_width = 250  # Adjust as needed based on content length

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
  # Cell for Company
  pdf.cell(cell_width, 10, txt=f"Company: {row['company']}", ln=True)

  # Cell for Date Published (can adjust width if needed for long dates)
  pdf.cell(cell_width, 10, txt=f"Date Published: {row['datePublished']}", ln=True)

  # Cell for Author
  pdf.cell(cell_width, 10, txt=f"Author: {row['author']}", ln=True)

  # Cell for Headline - Set bold for emphasis
  #pdf.set_font('DejaVu', 'B', 14)  # Bold font for Headline
  pdf.cell(cell_width, 10, txt=f"Headline: {row['headline']}", ln=True)
  #pdf.set_font('DejaVu', '', 14)  # Reset font back to normal

  # Cell for Description
  pdf.cell(cell_width, 10, txt=f"Description: {row['description']}", ln=True)

  # Cell for Article Body - You can split into multiple cells if very long
  pdf.multi_cell(cell_width, 10, txt=f"Article Body: {row['articleBody']}")

  # Cell for Tags
  pdf.cell(cell_width, 10, txt=f"Tags: {row['tags']}", ln=True)

  # Cell for URL
  pdf.cell(cell_width, 10, txt=f"URL: {row['url']}", ln=True)

  # Add an empty line for spacing
  pdf.cell(0, 10, txt="", ln=True)

# Save the PDF with name .pdf
pdf.output("output_file.pdf")

# If you want to download the file to your local system
from google.colab import files
files.download('output_file.pdf')


<ipython-input-60-3a4b42b0aa40>:9: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
<ipython-input-60-3a4b42b0aa40>:10: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, '', 0, 1, 'C')
<ipython-input-60-3a4b42b0aa40>:27: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 8)
<ipython-input-60-3a4b42b0aa40>:30: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
<ipython-input-60-3a4b42b0aa40>:39: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(cell_width, 10, txt=f"Company: {row['company']}", ln=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
# Install the FPDF library
!pip install fpdf

# Import the library
from fpdf import FPDF
import pandas as pd

# Define a class to create the PDF
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 8)
        self.cell(0, 10, '', 0, 1, 'C')

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

# Load your CSV file
df = pd.read_csv('/content/drive/MyDrive/ALLM_Project/RI_M1.csv')

# Create instance of FPDF class
pdf = PDF()

# Add a page
pdf.add_page()

# Set font to Arial, size 12
pdf.set_font('Arial', size=8)

# Add Unicode font support
pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
pdf.set_font('DejaVu', '', 8)

# Calculate max cell width for each column
max_widths = [pdf.get_string_width(str(max(df[col], key=lambda x: len(str(x))))) + 10 for col in df.columns]

# Add a cell for each field in each row
for index, row in df.iterrows():
    for i, col in enumerate(df.columns):
        pdf.multi_cell(max_widths[i], 10, txt=str(row[col]), align='L', border=1)
    pdf.ln(0)  # Add a line break for the next row

# Save the pdf with name .pdf
pdf.output("output_file.pdf")

# If you want to download the file to your local system
from google.colab import files
files.download('output_file.pdf')


<ipython-input-58-f0ce06e4c544>:11: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 8)
<ipython-input-58-f0ce06e4c544>:12: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, '', 0, 1, 'C')
<ipython-input-58-f0ce06e4c544>:29: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', size=8)
<ipython-input-58-f0ce06e4c544>:32: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
<ipython-input-58-f0ce06e4c544>:41: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.multi_cell(max_widths[i], 10, txt=str(row[col]), align='L', bor

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [62]:
# Import the library
from fpdf import FPDF
import pandas as pd

# Define a class to create the PDF
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 8)
        self.cell(0, 10, '', 0, 1, 'C')

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

# Load your CSV file
df = pd.read_csv('/content/drive/MyDrive/ALLM_Project/RI_M1.csv')

# Create instance of FPDF class
pdf = PDF()

# Add a page
pdf.add_page()

# Set font to Arial, size 12
pdf.set_font('Arial', size=8)

# Add Unicode font support
pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
pdf.set_font('DejaVu', '', 8)

# Calculate max cell width for each column
max_widths = [pdf.get_string_width(str(df[col].astype(str).max())) + 10 for col in df.columns]

# Add a cell for each field in each row
for index, row in df.iterrows():
    for i, col in enumerate(df.columns):
        pdf.multi_cell(max_widths[i], 10, txt=str(row[col]), align='L', border=1)
    pdf.ln(0)  # Add a line break for the next row

# Save the pdf with name .pdf
pdf.output("output_file.pdf")

# If you want to download the file to your local system
from google.colab import files
files.download('output_file.pdf')



<ipython-input-62-abd8eeb29d3a>:8: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 8)
<ipython-input-62-abd8eeb29d3a>:9: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, '', 0, 1, 'C')
<ipython-input-62-abd8eeb29d3a>:26: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', size=8)
<ipython-input-62-abd8eeb29d3a>:29: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
<ipython-input-62-abd8eeb29d3a>:38: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.multi_cell(max_widths[i], 10, txt=str(row[col]), align='L', borde

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [57]:
!pip install fpdf

from fpdf import FPDF
import pandas as pd

# Define a class to create the PDF
class PDF(FPDF):
  def header(self):
    self.set_font('Arial', 'B', 12)
    self.cell(0, 10, '', 0, 1, 'C')

  def footer(self):
    self.set_y(-15)
    self.set_font('Arial', 'I', 8)
    self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

# Load your CSV file (replace with your actual file path)
df = pd.read_csv('/content/drive/MyDrive/ALLM_Project/RI_M1.csv')

# Create instance of FPDF class
pdf = PDF()

# Add a page
pdf.add_page()

# Set font to Arial, '', 11 pts (adjust as needed)
pdf.set_font('Arial', '', 11)

# Add Unicode font support for special characters
pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
pdf.set_font('DejaVu', '', 14)

# Increase cell width to fit more text
cell_width = 250  # Adjust as needed based on content length

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
  # Cell for Company
  pdf.cell(cell_width, 10, txt=f"Company: {row['company']}", ln=True)

  # ... (other cells for remaining data)

  # Option 1: Split Article Body based on character count with line breaks
  body_text = row['articleBody']
  max_chars_per_cell = 200  # Adjust this value based on your content and cell width
  while body_text:
      content = body_text[:max_chars_per_cell]
      pdf.multi_cell(cell_width, 10, txt=content)
      body_text = body_text[max_chars_per_cell:]

  # Option 2: Split Article Body using pandas (uncomment and implement)
  #  # Define a function to split text into chunks
  #  def split_text(text, max_chars):
  #      chunks = [text[i:i+max_chars] for i in range(0, len(text), max_chars)]
  #      return chunks
  #
  #  # Split Article Body into multiple DataFrame columns (adjust column names)
  #  split_body = split_text(row['articleBody'], 150)  # Adjust max characters per chunk
  #  df_temp = pd.DataFrame({'Article Body Part 1': [split_body[0]], 'Article Body Part 2': [split_body[1] if len(split_body) > 1 else '']})
  #
  #  # Iterate through split DataFrame rows and add content to PDF
  #  for idx, body_part in df_temp.iterrows():
  #      pdf.multi_cell(cell_width, 10, txt=body_part['Article Body Part 1'])
  #      if len(split_body) > 1:
  #          pdf.multi_cell(cell_width, 10, txt=body_part['Article Body Part 2'])

  # Add an empty line for spacing
  pdf.cell(0, 10, txt="", ln=True)

# Save the PDF with name .pdf
pdf.output("output_file.pdf")
from google.colab import files
files.download('output_file.pdf')

<ipython-input-57-c08dce0d5241>:9: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
<ipython-input-57-c08dce0d5241>:10: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, '', 0, 1, 'C')
<ipython-input-57-c08dce0d5241>:27: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 11)
<ipython-input-57-c08dce0d5241>:30: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
<ipython-input-57-c08dce0d5241>:39: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(cell_width, 10, txt=f"Company: {row['company']}", ln=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/ALLM_Project/RI.csv')

filtered_df = df[df['datePublished'] != 'error']
filtered_df.to_csv('/content/drive/MyDrive/ALLM_Project/RI_M.csv')

In [ ]:
!pip install lxml
!pip install requests
!pip install beautifulsoup4
!pip install pdfkit
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import re
import pdfkit

In [ ]:
def get_blog_url(soup):
  div_=soup.find_all('div', attrs={'class': 'FL PR20'})
  url_list=[]
  for title in div_:
    href =title.find('a')['href']
    url_list.append("https://www.moneycontrol.com/"+href)
  return url_list

In [ ]:
def get_blog_content(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'lxml')
    all_scripts = soup.find_all('script', attrs={'type': 'application/ld+json'})

    # Extract the JSON content
    article_str = ''
    for script in all_scripts:
        raw_article_str = script.get_text().strip()
        if raw_article_str.startswith('{') and raw_article_str.endswith('}'):
            article_str = raw_article_str
            break

    if not article_str:
        print("No valid JSON content found in script tags")
        return None

    try:
        article_dict = json.loads(article_str)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return None

    # Process article data if JSON parsing successful
    all_tags = soup.find_all('div', attrs={'class': 'tags_first_line'})
    lst_all_tags = []
    for tag in all_tags:
        lst_all_tags.append(tag.get_text())

    tags = lst_all_tags[0].replace('TAGS:', '').replace('\n', '').split('#')[1:]
    tags = ','.join([tag.strip() for tag in tags])
    article_dict['tags'] = tags

    return article_dict



In [ ]:
def get_page_no(url, sc_id, page_no, next, year):
  request= requests.get(url)
  soup =BeautifulSoup(request.text, 'lxml')
  all_page_no =soup.find_all('div', attrs={'class': 'pages MR10 MT15'})
  if all_page_no:
        page_list = [i.text for i in all_page_no[0].find_all('a')]
        if page_list:
            if any(map(str.isdigit, page_list[-1])):
                return int(page_list[-1]), next
            else:
                next = next + 1
                page_no = int(page_list[-2])
                url = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=" + sc_id + "&scat=&pageno=" + str(
                    page_no) + "&next=" + str(next) + "&durationType=Y&Year=" + str(year) + "&duration=1&news_type="
                return get_page_no(url, sc_id, page_no, next, year)
    # If page numbers not found or if there's an error, return 0
  return 0, 0

In [ ]:
def save_company_data(url_="https://www.moneycontrol.com/stocks/company_info/stock_news.php?", sc_id=[], page_no=1, next=0, years=[]):
  articles_content = ''
  for company in sc_id:

    for year in years:
      print('year:', year)
      print('page_no:',page_no)
      print('next:', next)
      url=url_ + "sc id="+company+"&scat=&pageno="+str(page_no)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
      print('url:', url)

      max_page_no, max_next= get_page_no(url, company, page_no, next, year)
      max_next= max_next + 1

      for i in range(max_next):
        for j in range((i*10)+1, (i*10)+2):
          if j <= max_page_no:
            url_list=[]
            url=url_ + "sc_id="+company+"&scat=&pageno="+str(j)+"&next="+str(i)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
            request= requests.get(url)
            soup= BeautifulSoup(request.text, 'lxml')
            url_list= get_blog_url(soup)

            for url in url_list:
                try:
                            article_dict= get_blog_content(url)
                            print(company)
                            print(article_dict['datePublished'])
                            print(article_dict['author'])
                            print(article_dict['headline'])
                            print(article_dict['description'])
                            print(article_dict['articleBody'])
                            print(article_dict['tags'])
                            print(article_dict['url'])
                            print('------------------------------------------')

                            articles_content += f"<h1>{article_dict['headline']}</h1>"
                            articles_content += f"<p><strong>Company:</strong> {company}</p>"
                            articles_content += f"<p><strong>Date Published:</strong> {article_dict['datePublished']}</p>"
                            articles_content += f"<p><strong>Author:</strong> {article_dict['author']}</p>"
                            articles_content += f"<p><strong>Description:</strong> {article_dict['description']}</p>"
                            articles_content += f"<p><strong>Tags:</strong> {article_dict['tags']}</p>"
                            articles_content += f"<p><strong>URL:</strong> {article_dict['url']}</p>"
                            articles_content += f"<p>{article_dict['articleBody']}</p>"
                            articles_content += "<hr>"
                except IndexError:
                  print("")
                  continue
            else:
                        break

    pdfkit.from_string(articles_content, '/content/drive/MyDrive/ALLM_Project/all_articles.pdf')

In [73]:
!pip install fpdf

from fpdf import FPDF
import pandas as pd

# Define a class to create the PDF
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, '', 0, 1, 'C')

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

# Load your CSV file
df = pd.read_csv('/content/drive/MyDrive/ALLM_Project/RI_M1.csv')

# Create instance of FPDF class
pdf = PDF()

# Add a page
pdf.add_page()

# Set font to Arial, '', 12 pts
pdf.set_font('Arial', '', 10)

# Add Unicode font support for special characters
pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
pdf.set_font('DejaVu', '', 10)

# Adjust cell width and height to fit more content
cell_width = 180  # Adjust as needed based on content length
cell_height = 7   # Adjust as needed based on content length

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Cell for Company
    pdf.cell(cell_width, cell_height, txt=f"Company: {row['company']}", ln=True)

    # Cell for Date Published (can adjust width if needed for long dates)
    pdf.cell(cell_width, cell_height, txt=f"Date Published: {row['datePublished']}", ln=True)

    # Cell for Author
    pdf.cell(cell_width, cell_height, txt=f"Author: {row['author']}", ln=True)

    # Cell for Headline - Set bold for emphasis
    #pdf.set_font('DejaVu', 'B', 14)  # Bold font for Headline
    pdf.cell(cell_width, cell_height, txt=f"Headline: {row['headline']}", ln=True)
    #pdf.set_font('DejaVu', '', 14)  # Reset font back to normal

    # Cell for Description
    pdf.multi_cell(cell_width, cell_height, txt=f"Description: {row['description']}", ln=True)

    # Cell for Article Body - You can split into multiple cells if very long
    pdf.multi_cell(cell_width, cell_height, txt=f"Article Body: {row['articleBody']}")

    # Cell for Tags
    pdf.multi_cell(cell_width, cell_height, txt=f"Tags: {row['tags']}", ln=True)

    # Cell for URL
    pdf.cell(cell_width, cell_height, txt=f"URL: {row['url']}", ln=True)

    # Add an empty line for spacing
    pdf.cell(0, cell_height, txt="", ln=True)

# Save the PDF with name output_file.pdf
pdf.output("output_file.pdf")

# If you want to download the file to your local system
from google.colab import files
files.download('output_file.pdf')


<ipython-input-73-00763563ce93>:9: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
<ipython-input-73-00763563ce93>:10: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, '', 0, 1, 'C')
<ipython-input-73-00763563ce93>:27: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 10)
<ipython-input-73-00763563ce93>:30: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
<ipython-input-73-00763563ce93>:40: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(cell_width, cell_height, txt=f"Company: {row['company']}",

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [72]:
!pip install fpdf

from fpdf import FPDF
import pandas as pd

# Define a class to create the PDF
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, '', 0, 1, 'C')

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

# Load your CSV file
df = pd.read_csv('/content/drive/MyDrive/ALLM_Project/RI_M1.csv')

# Create instance of FPDF class
pdf = PDF()

# Add a page
pdf.add_page()

# Set font to Arial, '', 10 pts
pdf.set_font('Arial', '', 10)

# Add Unicode font support for special characters
pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
pdf.set_font('DejaVu', '', 10)

# Adjust cell width and height to fit more content
cell_width = 180  # Adjust as needed based on content length
cell_height = 7   # Adjust as needed based on content length

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Cell for Company
    pdf.cell(cell_width, cell_height, txt=f"Company: {row['company']}", ln=True)

    # Cell for Date Published (can adjust width if needed for long dates)
    pdf.cell(cell_width, cell_height, txt=f"Date Published: {row['datePublished']}", ln=True)

    # Cell for Author
    pdf.cell(cell_width, cell_height, txt=f"Author: {row['author']}", ln=True)

    # Cell for Headline - Set bold for emphasis
    #pdf.set_font('DejaVu', 'B', 14)  # Bold font for Headline
    pdf.cell(cell_width, cell_height, txt=f"Headline: {row['headline']}", ln=True)
    #pdf.set_font('DejaVu', '', 14)  # Reset font back to normal

    # Cell for Description
    pdf.multi_cell(cell_width, cell_height, txt=f"Description: {row['description']}", ln=True)

    # Cell for Article Body - You can split into multiple cells if very long
    pdf.multi_cell(cell_width, cell_height, txt=f"Article Body: {row['articleBody']}")

    # Cell for Tags
    tags_text = f"Tags: {row['tags']}"
    tags_lines = pdf.multi_cell(cell_width, cell_height, txt=tags_text, align='L')

    # Calculate the number of lines needed for Tags
    num_lines_tags = len(tags_lines) if isinstance(tags_lines, list) else 1

    # Cell for URL
    pdf.cell(cell_width, cell_height * num_lines_tags, txt=f"URL: {row['url']}", ln=True)

    # Add an empty line for spacing
    pdf.cell(0, cell_height, txt="", ln=True)

# Save the PDF with name output_file.pdf
pdf.output("output_file.pdf")

# If you want to download the file to your local system
from google.colab import files
files.download('output_file.pdf')




<ipython-input-72-399e498fcd31>:9: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
<ipython-input-72-399e498fcd31>:10: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, '', 0, 1, 'C')
<ipython-input-72-399e498fcd31>:27: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 10)
<ipython-input-72-399e498fcd31>:30: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('DejaVu', '', '/content/drive/MyDrive/ALLM_Project/DejaVuSansCondensed.ttf', uni=True)
<ipython-input-72-399e498fcd31>:40: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(cell_width, cell_height, txt=f"Company: {row['company']}",

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
save_company_data(sc_id=["RI"], page_no=1, next=0, years=[2017])

year: 2017
page_no: 1
next: 0
url: https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc id=RI&scat=&pageno=1&next=0&durationType=Y&Year=2017&duration=1&news_type=
RI


KeyError: 'datePublished'